# Loop Workflow Example 1

> * High level approach to making a 3D model from just a bounding box (in Australia only for now).

In [ ]:
import time
t0 = time.time()

## Map2Loop

In [ ]:
import os
from map2loop.project import Project

proj = Project( 
    loopdata_state="WA",
)

proj.update_config(
                    out_dir='./gmd-model',
                    overwrite='in-place',
                    bbox_3d={
                         "minx": 515687.31005864, # region of interest for GMD paper
                         "miny": 7473446.76593407,
                         "maxx": 562666.860106543,
                         "maxy": 7521273.57407786,
                         "base": -3200,
                         "top": 1200,
                     },
                    proj_crs={'init': 'EPSG:28350'},
                    quiet='all',
                  )
proj.config.c_l['intrusive']='granite'
proj.run()

## Loop Structural

In [ ]:
# Define project pathing from m2l
proj_path = proj.config.project_path
graph_path = proj.config.graph_path
tmp_path = proj.config.tmp_path
data_path = proj.config.data_path
dtm_path = proj.config.dtm_path
output_path = proj.config.output_path
vtk_path = proj.config.vtk_path

# Define project bounds
minx,miny,maxx,maxy = proj.config.bbox
model_base = proj.config.bbox_3d['base']
model_top = proj.config.bbox_3d['top']

fault_file = proj.config.fault_file_csv

In [ ]:
from LoopStructural import GeologicalModel
from LoopStructural.visualisation import LavaVuModelViewer
from datetime import datetime
import os
import time
from datetime import datetime
import shutil
import logging
logging.getLogger().setLevel(logging.ERROR)
import lavavu
import numpy as np

t1a = time.time()

nowtime=datetime.now().isoformat(timespec='minutes')   
model_name='leaflet'+'_'+nowtime.replace(":","-").replace("T","-")
if (os.path.exists(vtk_path+model_name)):
    shutil.rmtree(vtk_path+model_name)
os.mkdir(vtk_path+model_name)
filename=vtk_path+model_name+'/'+'surface_name_{}.vtk'


f=open(tmp_path+'/bbox.csv','w')
f.write('minx,miny,maxx,maxy,lower,upper\n')
ostr='{},{},{},{},{},{}\n'.format(minx,miny,maxx,maxy,model_base,model_top)
f.write(ostr)
f.close()


fault_params = {'interpolatortype':'FDI',
                 'nelements':1e5,
                'step':10,
                'fault_buffer':0.3,
                'solver':'cg',
                'cpw':10,
                'npw':10}
foliation_params = {'interpolatortype':'FDI' , # 'interpolatortype':'PLI',
                    'nelements':1e5,  # how many tetras/voxels
                    'buffer':1.8,  # how much to extend nterpolation around box
                    'solver':'cg',
                    'cpw':10,
                    'npw':10}
model, m2l_data = GeologicalModel.from_map2loop_directory(proj_path,
                                                          #    evaluate=False,
                                                          fault_params=fault_params,
                                                          rescale=False,
                                                          foliation_params=foliation_params,
                                                         skip_features=['Fault_16245'])
#model.to_file(output_path + "/model.pickle")    

view = LavaVuModelViewer(model,vertical_exaggeration=1) 
view.nsteps = np.array([200,200,200])
view.nsteps=np.array([50,50,50])
for sg in model.feature_name_index:
    if( 'super' in sg):
        view.add_data(model.features[model.feature_name_index[sg]])
view.nelements = 1e5
view.add_model_surfaces(filename=filename,faults=False)
view.nelements=1e6
view.add_model_surfaces(filename=filename,strati=False,displacement_cmap = 'rainbow')
view.lv.webgl(vtk_path+model_name)
view.nsteps = np.array([200,200,200])
view.add_model()

view.lv.control.Range('alpha', label="Global Opacity")
view.lv.control.DualRange(['xmin', 'xmax'], label="x clip", step=0.01, values=[0.0,1.0])
view.lv.control.DualRange(['ymin', 'ymax'], label="y clip", step=0.01, values=[0.0,1.0])
view.lv.control.DualRange(['zmin', 'zmax'], label="z clip", step=0.01, values=[0.0,1.0])
view.lv.control.Range(command='background', range=(0,1), step=0.1, value=0.8)
view.lv.control.show() #Show the control panel, including the viewer window
view.interactive()  

t2 = time.time()
print("m2l",(t1a-t0)/60.0,"LoopStructural",(t2-t1a)/60.0,"Total",(t2-t0)/60.0,"minutes")